In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv("/data/projects/processBio/ncbi/gene2refseq",delimiter="\t")

In [ ]:
rf = df[(df.status == "REVIEWED")
        & (df.assembly == "Reference GRCh38.p13 Primary Assembly")
        & (df["protein_accession.version"] != "-")
        & (df["RNA_nucleotide_accession.version"].str.contains("NM_"))]

In [ ]:
rf[rf.GeneID == 9381]

In [ ]:
summary = pd.read_csv("/data/projects/processBio/clinvar/clinvar/variant_summary.txt",delimiter="\t")

In [ ]:
summary = summary[(summary.Type == "single nucleotide variant") \
                  & (summary.ReviewStatus.isin(["criteria provided, single submitter",
              "criteria provided, multiple submitters, no conflicts",
              "reviewed by expert panel",
              "practice guideline",
              ])) & (summary.Assembly == "GRCh38")]

In [ ]:
summary = summary.assign(nmid=summary.Name.apply(lambda n: n[:n.find("(")]))

In [ ]:
summary

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
nmid2npid = {}
for nmid in tqdm(summary.nmid.unique()):
    nmid2npid[nmid] = rf[rf["RNA_nucleotide_accession.version"] == nmid]["protein_accession.version"].unique()

In [ ]:
nmid2npid

In [ ]:
summary = summary.assign(npid=summary.nmid.apply(lambda n: nmid2npid[n]))

In [ ]:
summary

In [ ]:
from Bio import Entrez
Entrez.email = 'zeiberg.d@northeastern.edu'

def getSequenceFromNPID(npid):
    "Return the protein sequence from "
    handle = Entrez.efetch(db="protein",id=npid, rettype="fasta", retmode="text")
    lines = handle.readlines()
    lines = [l.strip() for l in lines]
    return "".join(lines[1:])

In [ ]:
set([s[0] for s in summary.npid.values if len(s)])

In [ ]:
npid2seq = {}
for npid in tqdm(set([s[0] for s in summary.npid.values if len(s)])):
    try:
        npid2seq[npid[0]] = getSequenceFromNPID(npid[0])
    except Entrez.HTTPError:
        continue

In [ ]:
npid2seq